# OpenSearch Warming Up (기존 인덱스를 한글 형태소 분석기 사용하여 리인덱식)
>이 노트북은 SageMaker Studio* Data Science 3.0 kernel 및 ml.t3.medium 인스턴스에서 테스트 되었습니다.



여기서는 OpenSearch 가 설치된 것을 가정하고, 한글 형태소 분석기의 사용하는 법을 알려 드립니다.

---

### [중요]
- 이 노트북은 Bedrock Titan Embedding Model 을 기본으로 사용합니다. KoSIMCSERoberta 를 세이지 메이커 엔드포인트로 사용하신다면 아래의 선수 조건을 확인하세요.

#### 선수조건 (KoSIMCSERoberta 사용시)
- 임베딩 모델의 세이지 메이커 엔드포인트가 액티브 된 상태를 가정 합니다.
    - 세이지 메이커 엔드포인트에 배포하기 위해서는 아래 노트북을 실행하시고, Endpoint Name 만을 복사 하시면 됩니다.
    - [KoSIMCSERoberta Embedding Model 배포](https://github.com/gonsoomoon-ml/Kor-LLM-On-SageMaker/blob/main/1-Lab01-Deploy-LLM/4.Kor-Embedding-Model.ipynb)
    - SageMaker Endpoint 에 대해서는 공식 개발자 문서를 참조하세요 --> [Create your endpoint and deploy your model](https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints-deployment.html)
- 오픈 서치 서비스가 액티브 된 상태를 가정 합니다.


---
## Ref: 
- [Amazon OpenSearch Service로 검색 구현하기](https://catalog.us-east-1.prod.workshops.aws/workshops/de4e38cb-a0d9-4ffe-a777-bf00d498fa49/ko-KR/indexing/blog-reindex)
- [OpenSearch Python Client](https://opensearch.org/docs/1.3/clients/python-high-level/)
- [OpenSearch Match, Multi-Match, and Match Phrase Queries](https://opster.com/guides/opensearch/opensearch-search-apis/opensearch-match-multi-match-and-match-phrase-queries/)
- OpenSearch Query 에서 Filter, Must, Should, Not Mush 에 대한 설명 입니다.
    - [OpenSearch Boolean Queries](https://opster.com/guides/opensearch/opensearch-search-apis/opensearch-boolean-queries/#:~:text=Boolean%20queries%20are%20used%20to,as%20terms%2C%20match%20and%20query_string.)


# 0. 필요 패키지 설치

In [2]:
# install_needed = True  # should only be True once
install_needed = False  # should only be True once

In [3]:
# Make sure you ran `download-dependencies.sh` from the root of the repository first!
import sys
import IPython

if install_needed:
    print("installing deps and restarting kernel")
    !{sys.executable} -m pip install opensearch-py==2.3.1
    !{sys.executable} -m pip install langchain==0.0.279    
    !{sys.executable} -m pip install sqlalchemy==2.0.1
    
    IPython.Application.instance().kernel.do_shutdown(True)    
    

# 1. 환경 세팅

In [5]:
%load_ext autoreload
%autoreload 2

import sys, os
module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import print_ww

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Bedrock Client 생성
### 선수 지식
아래의 노트북을 먼저 실행해서, Bedrock 에 접근 가능하게 합니다.
- amazon-bedrock-workshop-webinar-kr/00_Setup/setup.ipynb

In [6]:
from utils.rag import get_embedding_model

In [7]:
import json
import boto3

from utils import bedrock, print_ww


# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
# os.environ["BEDROCK_ENDPOINT_URL"] = "<YOUR_ENDPOINT_URL>"  # E.g. "https://..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

Create new client
  Using region: us-east-1
  Using profile: None
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)


## Embedding 모델 선택

In [8]:
is_bedrock_embeddings = True
#is_bedrock_embeddings = False
# is_KoSimCSERobert = True
is_KoSimCSERobert = False

##############################
# Parameters for is_KoSimCSERobert
##############################
aws_region = os.environ.get("AWS_DEFAULT_REGION", None)
# KoSimCSE-roberta 를 사용한다면 
endpont_name = "KoSimCSE-roberta-2023-09-01-14-34-11"
##############################

llm_emb = get_embedding_model(boto3_bedrock, is_bedrock_embeddings, is_KoSimCSERobert, aws_region, endpont_name)    

Bedrock Embeddings Model Loaded


# 2. 데이터 준비


##  신한은행 FAQ 데이터 세트로 구현
- [중요] 저자 및 동료가 아래의 웹사이트에서 크로링한 기준으로 구성 하였습니다.
- 인터넷뱅킹 FAQ > 스마트뱅킹 No.1 ~ N. 89 로 구성되었습니다. 
- https://www.shinhan.com/hpe/index.jsp#050101020000

In [10]:
import pandas as pd
pd.options.display.max_rows = 20

data_file_path = "data/fsi_smart_faq_ko.csv"
df = pd.read_csv(data_file_path)
df

,no,Category,Information,type,Source
0,91,아마존 은행의 타기관OTP 이용등록방법 알려주세요,아마존 은행의 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 ...,인터넷뱅킹,아마존은행
1,90,아마존 공동인증서와 금융인증서 차이점이 무엇인가요?,공동인증서 (구 공인인증서)는 용도에 따라 은행/신용카드/보험용 무료 인증서와 전자...,인증서,아마존은행
2,88,공동인증서와 금융인증서 차이점이 무엇인가요?,공동인증서 (구 공인인증서)는 용도에 따라 은행/신용카드/보험용 무료 인증서와 전자...,인증서,신한은행
3,89,타기관OTP 이용등록방법 알려주세요,타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 이용가능합니다....,인터넷뱅킹,신한은행
4,88,공동인증서와 금융인증서 차이점이 무엇인가요?,공동인증서 (구 공인인증서)는 용도에 따라 은행/신용카드/보험용 무료 인증서와 전자...,인증서,신한은행
...,...,...,...,...,...
87,5,인터넷에서 이체한 거래의 상세내역 및 영수증을 인쇄하고 싶어요,"인터넷상에서 이체한 거래의 경우 인터넷상에서 제공하는 영수증은 없으며, 거래의 참고...",인터넷뱅킹,신한은행
88,4,타행으로 자동이체를 신청하고 싶은데요,인터넷뱅킹을 통한 타은행 계좌로의 자동이체(납부자 자동이체)신청이 가능하며 수수료는...,인터넷뱅킹,신한은행
89,3,공과금 자동이체 신청이 가능한가요?,"인터넷뱅킹에 로그인하신 후 ""공과금/법원 > 공과금센터"" 페이지에 가시면 ""지로자동...",인터넷뱅킹,신한은행
90,2,인터넷뱅킹 거래에 대한 책임 범위를 알고싶습니다.,기본적으로 은행이 인터넷뱅킹 사고에 대한 책임을 집니다. 해커의 침입에 의해 자금이...,인터넷뱅킹,신한은행


## 데이터 전처리
- 여기서 no 는 제거 합니다. 

In [11]:
os.makedirs("data", exist_ok=True)

In [12]:
def preprocess_data(df):
    ldf = df.copy()

    ldf.rename(columns={'Category': 'ask'}, inplace=True)
    df_index = ldf.drop(['no'], axis=1)
    # df_index.to_csv("rag_data_kr/fsi_smart_faq_ko_answer.csv", index=None, header=None)
    df_index.to_csv("data/fsi_smart_faq_ko_preprocess.csv", index=None)

    return df_index

pre_df = preprocess_data(df)
pre_df.head(3)

,ask,Information,type,Source
0,아마존 은행의 타기관OTP 이용등록방법 알려주세요,아마존 은행의 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 ...,인터넷뱅킹,아마존은행
1,아마존 공동인증서와 금융인증서 차이점이 무엇인가요?,공동인증서 (구 공인인증서)는 용도에 따라 은행/신용카드/보험용 무료 인증서와 전자...,인증서,아마존은행
2,공동인증서와 금융인증서 차이점이 무엇인가요?,공동인증서 (구 공인인증서)는 용도에 따라 은행/신용카드/보험용 무료 인증서와 전자...,인증서,신한은행


### CSVLoader 로 문서 로딩

In [13]:
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import FAISS
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter

In [14]:
loader = CSVLoader(
    file_path="data/fsi_smart_faq_ko_preprocess.csv",
    # csv_args={
    #     "delimiter": ",",
    #     "fieldnames": ["Category", "Information", "type", "Source"],
    # },    
    source_column="Source",
    encoding="utf-8"
)
import time
documents_fsi = loader.load()


In [15]:
print(len(documents_fsi))

documents_fsi[0]
# documents_fsi[-1]

92


Document(page_content='ask: 아마존 은행의 타기관OTP 이용등록방법 알려주세요\nInformation: 아마존 은행의 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 이용가능합니다. \r\n[경로]\r\n- 인터넷뱅킹 로그인→ 사용자관리→인터넷뱅킹관리→OTP이용등록  \r\n- 아마존은행 쏠(SOL) 로그인→ 전체메뉴→설정/인증→ 이용중인 보안매체선택→   OTP이용등록\r\n \r\n ※ OTP이용등록후 재로그인을 하셔야 새로 등록된 보안매체가 적용됩니다.\r\n\r\n기타 궁금하신 내용은 아마존 은행 고객센터 1599-9999로 문의하여 주시기 바랍니다.\ntype: 인터넷뱅킹\nSource: 아마존은행', metadata={'source': '아마존은행', 'row': 0})

## Text Spliter 로 청킹
참고: 검색된 문서/텍스트는 질문에 대답하기에 충분한 정보를 포함할 만큼 커야 합니다. 하지만 LLM 프롬프트에 들어갈 만큼 충분히 작습니다. 또한 임베딩 모델에는 입력 토큰 길이가 512개 토큰으로 제한되어 있으며 이는 한국어의 경우에 대략 180 ~ 200 글자로 변환됩니다. 이 사용 사례를 위해 [RecursiveCharacterTextSplitter](https://python.langchain.com/en/latest/modules/indexes/text_splitters/examples/recursive_text_splitter.html)를 사용하여 210자가 겹치는 약 50자의 청크를 생성합니다.

In [16]:
if is_bedrock_embeddings:
    chunk_size = 2048
    chunk_overlap = 0
elif is_KoSimCSERobert:
    chunk_size = 800 # This is maxumum
    chunk_overlap = 0
    

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = chunk_size,
    chunk_overlap  = chunk_overlap,
    separators=["\n\n", "\n", ".", " ", ""],
    length_function = len,
)

docs = text_splitter.split_documents(documents_fsi)
print(f"Number of documents after split and chunking={len(docs)}")

Number of documents after split and chunking=92


# 3. OpenSearch Client 생성
### 선수 조건
- 아래의 링크를 참조해서 OpenSearch Service 를 생성하고, opensearch_domain_endpoint, http_auth 를 복사해서, 아래 셀의 내용을 대체 하세요.
    - [OpenSearch 생성 가이드](https://github.com/gonsoomoon-ml/Kor-LLM-On-SageMaker/blob/main/2-Lab02-QA-with-RAG/4.rag-fsi-data-workshop/TASK-4_OpenSearch_Creation_and_Vector_Insertion.ipynb)
### 아래 셀에 다음의 정보가 입력이 되어야 합니다.
```
opensearch_domain_endpoint = "<Type Domain Endpoint>"
http_auth = (rag_user_name, rag_user_password) # Master username, Master password

```


In [17]:

from utils.rag import create_aws_opensearch_client, check_if_index_exists, delete_index
# from utils.rag import create_index, add_doc, search_document

In [18]:
aws_region = 'us-east-1'
opensearch_cluster_domain = 'https://search-gonsoo-jeiuxgzz4322ulsrbokvnqispu.us-east-1.es.amazonaws.com'

os.environ["OpenSearch_UserName"] = "<Type UserName>" 
os.environ["OpenSearch_UserPassword"] = "<Type Password>" 

rag_user_name = os.environ["OpenSearch_UserName"]
rag_user_password = os.environ["OpenSearch_UserPassword"]

# OpenSearch Dashboards URL = https://search-gonsoo-jeiuxgzz4322ulsrbokvnqispu.us-east-1.es.amazonaws.com/_dashboards
opensearch_domain_endpoint = "https://search-gonsoo-jeiuxgzz4322ulsrbokvnqispu.us-east-1.es.amazonaws.com"
http_auth = (rag_user_name, rag_user_password) # Master username, Master password



aws_client = create_aws_opensearch_client(
                                     aws_region,
                                     opensearch_domain_endpoint,
                                     http_auth)



# 4. OpenSearch 벡터 Indexer 생성
### 선수 조건
- 아래의 링크를 참조해서 OpenSearch Service 를 생성하고, opensearch_domain_endpoint, http_auth 를 복사해서, 아래 셀의 내용을 대체 하세요.
    - [OpenSearch 생성 가이드](https://github.com/gonsoomoon-ml/Kor-LLM-On-SageMaker/blob/main/2-Lab02-QA-with-RAG/4.rag-fsi-data-workshop/TASK-4_OpenSearch_Creation_and_Vector_Insertion.ipynb)
- 랭체인 오프서처 참고 자료
    - [Langchain Opensearch](https://python.langchain.com/docs/integrations/vectorstores/opensearch)

## 오픈 서치 인덱스 유무에 따라 삭제
오픈 서치에 해당 인덱스가 존재하면, 삭제 합니다. 

In [19]:
from utils.rag import create_aws_opensearch_client, check_if_index_exists, delete_index
from utils.rag import create_index, search_document

aws_client = create_aws_opensearch_client(
                                     aws_region,
                                     opensearch_domain_endpoint,
                                     http_auth)


index_name = "fsi-v10"
index_exists = check_if_index_exists(aws_client, index_name)

if index_exists:
    delete_index(aws_client, index_name)    
else:
    print("Index does not exist")

index_name=fsi-v10, exists=True

Deleting index:
{'acknowledged': True}


## 인덱스 생성

In [20]:
%%time

# by default langchain would create a k-NN index and the embeddings would be ingested as a k-NN vector type
from langchain.vectorstores import OpenSearchVectorSearch
docsearch = OpenSearchVectorSearch.from_documents(
    index_name=index_name,
    documents=docs,
    embedding=llm_emb,
    opensearch_url=opensearch_domain_endpoint,
    http_auth=http_auth,
    bulk_size=10000,
    timeout=60
)

CPU times: user 303 ms, sys: 45.2 ms, total: 348 ms
Wall time: 9.62 s


## 인덱싱된 벡터 확인

- [langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch.html)

In [21]:
vectro_db = OpenSearchVectorSearch(
    index_name=index_name,
    opensearch_url=opensearch_domain_endpoint,
    embedding_function=llm_emb,
    http_auth=http_auth, # http_auth
    is_aoss =False,
    engine="faiss",
    space_type="l2"
)

# 5. 오픈 서치에 "유사 서치" 검색
- query 를 제공해서 실제로 유사한 내용이 검색이 되는지를 확인 합니다.



- similarity_search_with_score API 정보
    - [API: similarity_search_with_score](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch.html#langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch.similarity_search)

In [22]:
from langchain import PromptTemplate
from langchain.chains.question_answering import load_qa_chain

/opt/conda/lib/python3.10/site-packages/langchain/__init__.py:24: UserWarning: Importing PromptTemplate from langchain root module is no longer supported.
  warnings.warn(


In [23]:
from utils.rag import opensearch_pretty_print_documents_wo_filter        

def run_similarity_search_with_score_filter(query, is_filter, boolean_filter, k=5):
    if is_filter:
        pre_similar_doc = vectro_db.similarity_search_with_score(
            query,
            k=k,
            # fetch_k=3,
            search_type="approximate_search", # approximate_search, script_scoring, painless_scripting
            space_type="l2",     #"l2", "l1", "linf", "cosinesimil", "innerproduct", "hammingbit";
            boolean_filter= boolean_filter
        )
    else:
        pre_similar_doc = vectro_db.similarity_search_with_score(
            query,
            k=k,
            # fetch_k=3,
            search_type="approximate_search", # approximate_search, script_scoring, painless_scripting
            space_type="l2",     #"l2", "l1", "linf", "cosinesimil", "innerproduct", "hammingbit";
        )
        
    
    return pre_similar_doc



In [24]:
# query = '간편조회서비스는 회원가입해야하나요?'
query = "타기관OTP 등록 방법 알려주세요"

# filter01 = "공과금"

# filter01 = "인터넷뱅킹"
pre_similar_doc = run_similarity_search_with_score_filter(query, is_filter=False, boolean_filter=None, k=3)

opensearch_pretty_print_documents_wo_filter(pre_similar_doc)


Score: 0.0051887482
Document Number: 3
ask: 타기관OTP 이용등록방법 알려주세요
Information: 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 이용가능합니다. 
type: 인터넷뱅킹
Source: 신한은행
--------------------------------------------------

Score: 0.0044330047
Document Number: 0
ask: 아마존 은행의 타기관OTP 이용등록방법 알려주세요
Information: 아마존 은행의 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 이용가능합니다. 
type: 인터넷뱅킹
Source: 아마존은행
--------------------------------------------------

Score: 0.004250728
Document Number: 11
ask: 인터넷뱅킹에서 모바일OTP 인증요청 했는데 푸시메세지가 오지 않아요. 어떻게 해야하나요?
Information: 신한 쏠(SOL) 푸시알림 수신에 동의를 하신경우 푸시를 받을수 있습니다. 경로 
type: 인터넷뱅킹
Source: 신한은행
--------------------------------------------------


# 6. 키워드 검색

## 'Title" 에 "출시" 단어를 검색합니다.

In [25]:
from utils.rag import generate_opensearch_AndQuery, parse_keyword_response


In [27]:

def run_keyword_query(q, aws_client, index_name):
    # opensearch_query = generate_opensearch_AndQuery(q)
    # print(opensearch_query)    
    query = {
      "query": {
        "match": {
          "text": {
            "query": f"{q}"
          }
        }
      }
    }    
    print("query: ", query)
    
    response = search_document(aws_client, query, index_name)
    return response

            
q = '인터넷뱅킹'
response = run_keyword_query(q, aws_client, index_name)
parse_keyword_response(response, show_size=10)

query:  {'query': {'match': {'text': {'query': '인터넷뱅킹'}}}}

Search results:
# of searched docs:  10
# of display: 10
---------------------
_id in index:  b6682ee0-4da2-4ccd-b96c-acd810f73f79
1.3649293
ask: 영업점에서 인터넷뱅킹 가입하고 왔는데 홈페이지회원은 별도로 가입해야 하나요?
Information: 인터넷뱅킹 가입과 홈페이지 회원은 개별로 운영되고 있습니다. 
인터넷뱅킹만 가입을 하셨다면 별도로 홈페이지에 회원가입 후 홈페이지 로그인이 가능합니다. 
※ 영업점에서 인터넷뱅킹 가입 시 전계좌조회서비스 가입하신 고객은 홈페이지 상에서 이용자비밀번호 등록 후 이용가능합니다. 
기타 궁금하신 내용은 신한은행 고객센터 1599-8000로 문의하여 주시기 바랍니다.
type: 홈페이지
Source: 신한은행
---------------------
_id in index:  8dd7ecf9-bb5a-4989-88b6-d9d13eab7385
1.334384
ask: 이메일주소변경 안내
Information: 이메일주소 변경방법은 아래와 같습니다. ① 인터넷뱅킹 로그인 ② 사용자관리 고객정보 고객정보조회/변경메뉴 클릭 ③ 고객님의 다른 정보와 함께 이메일정보 변경 가능
type: 인터넷뱅킹
Source: 신한은행
---------------------
_id in index:  1bb3f2cd-5fa9-4878-b19a-c726a0b2b329
1.2610028
ask: 12개월 장기미이체로 이체서비스가 중지된 경우 어떻게 해제하나요?
Information: 최근 12개월동안 인터넷뱅킹/폰뱅킹/모바일뱅킹에서 이체서비스를 이용하지 않은 경우 고객님의 금융자산 보호를 위하여 이체서비스가 중단됩니다. 다시 이체를 원하시는 경우에는 인터넷뱅킹/폰뱅킹/신한쏠(SOL)에서 이체정지 해제를 하실 수 있습니다.(단, 인터넷뱅킹

In [28]:
q = '인터넷뱅킹'
keywords = q.split(' ')
print("keywords: \n", keywords)

keywords: 
 ['인터넷뱅킹']


In [30]:
# # Generate the OpenSearch query
# opensearch_query = generate_opensearch_AndQuery(keywords)
# print(opensearch_query)


In [31]:
q = '뱅킹'
response = run_keyword_query(q, aws_client, index_name)    
parse_keyword_response(response)

query:  {'query': {'match': {'text': {'query': '뱅킹'}}}}

Search results:
There is no response


In [32]:
_id = 'ebb7fed3-af3f-498d-a810-1ca4b52f77a2'
aws_client.termvectors(index=index_name, id=_id, fields='text')

{'_index': 'fsi-v10',
 '_id': 'ebb7fed3-af3f-498d-a810-1ca4b52f77a2',
 '_version': 0,
 'found': False,
 'took': 0}

# 7. 은전한닢 형태소 분석기를 포함한 New Index 생성 및 실험

In [33]:
new_index_name = "korea-fsi-v10"
index_exists = check_if_index_exists(aws_client, new_index_name)

if index_exists:
    delete_index(aws_client, new_index_name)    
else:
    print("Index does not exist")    
        
# Create an index with non-default settings.

index_body = {
    "settings": {
        "index": {
            "number_of_shards" : 3,
            "number_of_replicas": 1,
            "knn": True,
            "knn.algo_param": {
                "ef_search": 512
            }
        },
        "analysis": {
            "tokenizer": {
                "seunjeon": {
                    "type": "seunjeon_tokenizer"
                }
            },
            "analyzer": {
                "my_analyzer": {
                    "type": "custom",
                    "tokenizer": "seunjeon"
                }
            }
        }
    },
    "mappings": {
        "properties": {
            "text": {
                "type": "text",
                "fields": {
                    "keyword": {
                        "type": "keyword",
                        "ignore_above": 1024
                    }
                },
                "analyzer": "my_analyzer",
                "search_analyzer": "my_analyzer"        
            },
        "vector_field": {
            "type": "knn_vector",
            "dimension": 1536,
            "method": {
                "engine": "nmslib",
                "space_type": "l2",
                "name": "hnsw",
                "parameters": {
                    "ef_construction": 512,
                    "m": 16
                }
            }
        }
    }
  }
}

create_index(aws_client, new_index_name, index_body)
index_info = aws_client.indices.get(index=new_index_name)
index_info

index_name=korea-fsi-v10, exists=True

Deleting index:
{'acknowledged': True}

Creating index:
{'acknowledged': True, 'shards_acknowledged': True, 'index': 'korea-fsi-v10'}


{'korea-fsi-v10': {'aliases': {},
  'mappings': {'properties': {'text': {'type': 'text',
     'fields': {'keyword': {'type': 'keyword', 'ignore_above': 1024}},
     'analyzer': 'my_analyzer'},
    'vector_field': {'type': 'knn_vector',
     'dimension': 1536,
     'method': {'engine': 'nmslib',
      'space_type': 'l2',
      'name': 'hnsw',
      'parameters': {'ef_construction': 512, 'm': 16}}}}},
  'settings': {'index': {'number_of_shards': '3',
    'knn.algo_param': {'ef_search': '512'},
    'provided_name': 'korea-fsi-v10',
    'knn': 'true',
    'creation_date': '1695797238226',
    'analysis': {'analyzer': {'my_analyzer': {'type': 'custom',
       'tokenizer': 'seunjeon'}},
     'tokenizer': {'seunjeon': {'type': 'seunjeon_tokenizer'}}},
    'number_of_replicas': '1',
    'uuid': 'n61lDvghR6W8ADnTLFsk3w',
    'version': {'created': '136287827'}}}}}

## Re-indexing

In [34]:
index_name

'fsi-v10'

In [35]:
_reindex = {
  "source": {
    "index": index_name
  },
  "dest": {
    "index": new_index_name
  }
}

print("_reindex: \n", _reindex)

_reindex: 
 {'source': {'index': 'fsi-v10'}, 'dest': {'index': 'korea-fsi-v10'}}


In [36]:
aws_client.reindex(_reindex)

{'took': 754,
 'timed_out': False,
 'total': 92,
 'updated': 0,
 'created': 92,
 'deleted': 0,
 'batches': 1,
 'version_conflicts': 0,
 'noops': 0,
 'retries': {'bulk': 0, 'search': 0},
 'throttled_millis': 0,
 'requests_per_second': -1.0,
 'throttled_until_millis': 0,
 'failures': []}

In [37]:
q = '뱅킹'
response = run_keyword_query(q, aws_client, new_index_name)    
parse_keyword_response(response)

query:  {'query': {'match': {'text': {'query': '뱅킹'}}}}

Search results:
# of searched docs:  10
# of display: 3
---------------------
_id in index:  eeb05c3d-e350-488a-aea0-267a87592767
0.92703587
ask: 인터넷뱅킹 해지 방법 알려주세요
Information: [경로] - 인터넷뱅킹 로그인 → 사용자관리 → 인터넷뱅킹관리 → 인터넷뱅킹해지 ☞ 신한은행에서 발급한 인증서 폐기여부 선택가능합니다. ☞ 쏠(모바일뱅킹) 도 사용할수 없음 - 신한 쏠(SOL) 로그인 → 전체메뉴 → 설정/인증 → 서비스 해지 ☞ 쏠(SOL)만 해지 또는 인터넷뱅킹&쏠(SOL) 모두 해지 중 선택하여 해지가능합니다. 기타 궁금하신 내용은 신한은행 고객센터 1599-8000로 문의하여 주시기 바랍니다.
type: 인터넷뱅킹
Source: 신한은행
---------------------
_id in index:  1bb3f2cd-5fa9-4878-b19a-c726a0b2b329
0.9052478
ask: 12개월 장기미이체로 이체서비스가 중지된 경우 어떻게 해제하나요?
Information: 최근 12개월동안 인터넷뱅킹/폰뱅킹/모바일뱅킹에서 이체서비스를 이용하지 않은 경우 고객님의 금융자산 보호를 위하여 이체서비스가 중단됩니다. 다시 이체를 원하시는 경우에는 인터넷뱅킹/폰뱅킹/신한쏠(SOL)에서 이체정지 해제를 하실 수 있습니다.(단, 인터넷뱅킹 출금계좌가 등록되어 있는 경우) [경로] ① 당행 고객정보에 등록되어있는 연락처로 ARS 추가인증 가능한 경우: - 인터넷뱅킹 로그인→ 사용자관리→ 인터넷뱅킹관리→ 장기미사용 정지해제 (또는) - 쏠(SOL) 로그인→ 전체메뉴→ 이체 → 이체관리→ 장기미사용정지 해제 ② ARS 추가인증이 불가한 경우 (해외체류 고객등) : - 인터넷뱅킹에서는 해제 어려우며, 쏠 (SOL)에서 해제 가능합니다.

In [38]:
q = '인터넷 뱅킹'
response = run_keyword_query(q, aws_client, new_index_name)    
parse_keyword_response(response)

query:  {'query': {'match': {'text': {'query': '인터넷 뱅킹'}}}}

Search results:
# of searched docs:  10
# of display: 3
---------------------
_id in index:  eeb05c3d-e350-488a-aea0-267a87592767
1.7540421
ask: 인터넷뱅킹 해지 방법 알려주세요
Information: [경로] - 인터넷뱅킹 로그인 → 사용자관리 → 인터넷뱅킹관리 → 인터넷뱅킹해지 ☞ 신한은행에서 발급한 인증서 폐기여부 선택가능합니다. ☞ 쏠(모바일뱅킹) 도 사용할수 없음 - 신한 쏠(SOL) 로그인 → 전체메뉴 → 설정/인증 → 서비스 해지 ☞ 쏠(SOL)만 해지 또는 인터넷뱅킹&쏠(SOL) 모두 해지 중 선택하여 해지가능합니다. 기타 궁금하신 내용은 신한은행 고객센터 1599-8000로 문의하여 주시기 바랍니다.
type: 인터넷뱅킹
Source: 신한은행
---------------------
_id in index:  1bb3f2cd-5fa9-4878-b19a-c726a0b2b329
1.685121
ask: 12개월 장기미이체로 이체서비스가 중지된 경우 어떻게 해제하나요?
Information: 최근 12개월동안 인터넷뱅킹/폰뱅킹/모바일뱅킹에서 이체서비스를 이용하지 않은 경우 고객님의 금융자산 보호를 위하여 이체서비스가 중단됩니다. 다시 이체를 원하시는 경우에는 인터넷뱅킹/폰뱅킹/신한쏠(SOL)에서 이체정지 해제를 하실 수 있습니다.(단, 인터넷뱅킹 출금계좌가 등록되어 있는 경우) [경로] ① 당행 고객정보에 등록되어있는 연락처로 ARS 추가인증 가능한 경우: - 인터넷뱅킹 로그인→ 사용자관리→ 인터넷뱅킹관리→ 장기미사용 정지해제 (또는) - 쏠(SOL) 로그인→ 전체메뉴→ 이체 → 이체관리→ 장기미사용정지 해제 ② ARS 추가인증이 불가한 경우 (해외체류 고객등) : - 인터넷뱅킹에서는 해제 어려우며, 쏠 (SOL)에서 해제 가능합니

In [39]:
_id = 'cee4de4a-8921-4af9-b599-55fb90b27506'
aws_client.termvectors(index=new_index_name, id=_id, fields='text')

{'_index': 'korea-fsi-v10',
 '_id': 'cee4de4a-8921-4af9-b599-55fb90b27506',
 '_version': 0,
 'found': False,
 'took': 0}